## Initialize Main Functions and Global Variables

In [1]:
#Import relevant Classes and make matplotlib inline to avoid loading plots at runtime
import geojson
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import functools
import pandas as pd
import pickle as pkl
import time
from gzip import compress, decompress
from ipyleaflet import GeoJSON
from ipywidgets import Button, VBox, HBox, Box, Layout, Output, Text, Label
import sys
sys.path.append('../Predicted_Datasets/')
import folium
import os
import ee
#ee.Authenticate()
ee.Initialize()

In [2]:
#Set a number of global parameters for the entire form, edit this to specifications
#Number of images to shown at one time
IMGSHOWN=30
#Number of tile pairs to show per row
NUMIMGROW=3
#Name of the current dataset, change as needed
datasetName='googletest_state_sampled_slender_panda'
#File Path
pathway='Temp Images/'+datasetName+'/'

In [3]:
def loadGroup(imageDict,hrImgDict,groups,m,userNameValue,confirmed,denied,edit,add):
    #Create Button Click Events
    def confirmKiln(b,buttons,kilnType):
        for button in buttons:
            if button.style.button_color != 'gray':
                button.style.button_color='gray'
                
        if (b.tooltip in denied.value.keys()):
            edit.value[b.tooltip]={'add':kilnType,'data':{"time":time.time(),"user":userNameValue.value,"kilnType":kilnType}}
        elif b.tooltip in edit.value:
            edit.value[b.tooltip]={'add':kilnType,'data':{"time":time.time(),"user":userNameValue.value,"kilnType":kilnType}}
        elif (b.tooltip not in confirmed.value.keys()):
            add.value[b.tooltip]={'add':kilnType,'data':{"time":time.time(),"user":userNameValue.value,"kilnType":kilnType}}
        b.style.button_color='green'
            
    def denyKiln(b,buttons):
        for button in buttons:
            if button.style.button_color != 'gray':
                button.style.button_color='gray'
                
        if (b.tooltip in confirmed.value.keys()):
            edit.value[b.tooltip]={'add':'Denied','data':{"time":time.time(),"user":userNameValue.value}}
        elif (b.tooltip in edit.value):
            edit.value[b.tooltip]={'add':'Denied','data':{"time":time.time(),"user":userNameValue.value}}
        elif (b.tooltip not in denied.value.keys()):
            add.value[b.tooltip]={'add':'Denied','data':{"time":time.time(),"user":userNameValue.value}}
        b.style.button_color='red'
        
    def centerImage(b):
        m.location=(imageDict[b.tooltip][2].y,imageDict[b.tooltip][2].x)
    
    #Initialize return GJS and main Image display lists
    groupsGJS=[]
    groupsImg=[]
    
    #Iterate through the entire groups list and add the images and gjs to their respective position
    for group in range(len(groups.keys())):
        print("Adding group #",group," to the web app")
        #Initialize/Reset all relevant lists
        batchesGJS=[]
        batchGJS=[]
        batchesImg=[]
        batchImg=[]
        rowImg=[]
        limit=0
        for key in groups[str(group)]:
            #Limit reached, add the current batch images and gjs to the respective batch list and reset for next batch
            if limit == IMGSHOWN:
                batchImg.append(HBox(rowImg))
                batchesImg.append(VBox(batchImg))
                batchesGJS.append(batchGJS)
                batchGJS=[]
                batchImg=[]
                rowImg=[]
                limit=0
            
            #Load GJS
            folium.GeoJson(ee.Geometry(eval(imageDict[key][1])['geometry']).transform("EPSG:4326",1).getInfo(),name='tile'+str(key)).add_to(m)
            
            #Add and reset row if row is equal to NUMROWIMG
            if (len(rowImg) >= NUMIMGROW):
                batchImg.append(HBox(rowImg))
                rowImg=[]
                
            #Initialize Buttons and tie their respective click events
            confirmSquareButton = Button(
                description='Confirm Square Kiln',
                tooltip=str(key)
            )
            confirmCircleButton = Button(
                description='Confirm Circle Kiln',
                tooltip=str(key)
            )
            confirmBothButton = Button(
                description='Confirm Both Kilns',
                tooltip=str(key)
            )
            denyButton = Button (
                description="No Kiln",
                tooltip=str(key)
            )
            
            if key in confirmed.value.keys():
                if confirmed.value[key]['kilnType']=='Square':
                    confirmSquareButton.style.button_color='green'
                elif confirmed.value[key]['kilnType']=='Circle':
                    confirmCircleButton.style.button_color='green'
                else:
                    confirmBothButton.style.button_color='green'
                    
            if key in denied.value.keys():
                denyButton.style.button_color='red'
                
            centerButton = Button (
                description="Center to Image",
                tooltip=key
            )

            confirmSquareButton.on_click(functools.partial(confirmKiln,buttons=[confirmCircleButton,confirmBothButton,denyButton],kilnType='Square'))
            confirmCircleButton.on_click(functools.partial(confirmKiln,buttons=[confirmSquareButton,confirmBothButton,denyButton],kilnType='Circle'))
            confirmBothButton.on_click(functools.partial(confirmKiln,buttons=[confirmSquareButton,confirmCircleButton,denyButton],kilnType='Both'))
            denyButton.on_click(functools.partial(denyKiln,buttons=[confirmSquareButton,confirmCircleButton,confirmBothButton]))
            centerButton.on_click(centerImage)    
            
            #Create Output for the matplotlib plots and tie them to the respective image plot
            output=Output(layout={'border': '1px solid black'})
            output2=Output(layout={'border':'1px solid black'})
            with output:
                fig = plt.figure(figsize=(6,6))
                hr=plt.imshow(hrImgDict[key][0])
                plt.show(fig)
            with output2:
                fig2=plt.figure(figsize=(6,6))
                lr=plt.imshow(imageDict[key][0])
                plt.show(fig2)
            
            #Consolidate and order the Images and Buttons into a VBox and increase the limit
            rowImg.append(VBox([HBox([output,output2]),HBox([confirmSquareButton,confirmCircleButton,confirmBothButton,denyButton,centerButton])]))
            limit+=1
            
        #Do one final append to add any straggling gjs and images and add the total batch set to the respective group list
        batchesGJS.append(batchGJS)
        groupsGJS.append(batchesGJS)
        batchImg.append(HBox(rowImg))
        batchesImg.append(VBox(batchImg))
        groupsImg.append(batchesImg)
    return groupsGJS,groupsImg

In [4]:
def InitiateUI(confirmed,denied,groups,batch):
    #Clear existing plts
    plt.clf()
    
    #load image data from storage
    print("Loading Data")
    lrfile=open(pathway+datasetName+"LrBatch"+str(batch),'r+b')
    hrfile=open(pathway+datasetName+"HrBatch"+str(batch),'r+b')
    imageDict=pkl.load(lrfile)
    hrImgDict=pkl.load(hrfile)
    lrfile.close()
    hrfile.close()
      
    #initialize Map
    def add_ee_layer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
            name=name,
            overlay=True,
            control=True
        ).add_to(self)

    
    folium.Map.add_ee_layer = add_ee_layer
    m = folium.Map()
    
    class TextBox:
        def __init__(self,initial='noUser'):
            self.value=initial
        def changeName(self,text):
            self.value=text
            return;
        
    userNameValue=TextBox()
    
    class updateList:
        def __init__(self,initial=[]):
            self.value=initial
            
    confirmedClass=updateList(confirmed)
    deniedClass=updateList(denied)
    
    class editRequest:
        def __init__(self):
            self.value={}
    class addRequest:
        def __init__(self):
            self.value={}
    
    edit=editRequest()
    add=addRequest()
    
    #iterate through each cluster group and pre-load all the UI and return the lists
    print("Initializing UI!")
    groupGJS,groupImg=loadGroup(imageDict,hrImgDict,groups,m,userNameValue,confirmedClass,deniedClass,edit,add)
    print('Done!')
    
    #Initialize counter class and two counters to iteratively keep track between on-click even
        
    class Counter:
        def __init__(self, initial=0):
            self.value = initial

        def increment(self, amount=1):
            self.value += amount
            return self.value

        def reset(self):
            self.value=0

        def __iter__(self, sentinal=False):
            return iter(self.increment, sentinal)

    groupcounter=Counter()
    batchcounter=Counter()
    
    #create buttons and textBox to insert to our layout
    nextGroup=Button(
        description="Next Group",
        tooltip="Proceed"
    )
    
    saveData=Button(
        description="Save Data",
        tooltip="Save"
    )
    
    userName=Text(
        placeholder='Type your name!',
        description='User Name',
        disabled=False
    )
    
    submitName=Button(
        description="Submit",
        tooltip="Submit"
    )
    

    curLabel=Label(value="User: "+userNameValue.value+", Working on: Group: "+str(groupcounter.value)+", Batch: "+str(batchcounter.value))

    
    #On-Click call the next batch of gjs and images or the next group if the batches are done for that group
    def saveDataFunc(confirmed,denied,add,edit,b):
        print('Saving')
        newconfirmedFile=open(pathway+datasetName+'Confirmed','r+b')
        newdeniedFile=open(pathway+datasetName+'Denied','r+b')
        newConfirmed=pkl.load(newconfirmedFile)
        newDenied=pkl.load(newdeniedFile)
        newconfirmedFile.close()
        newdeniedFile.close()
        for req in add.value:
            if add.value[req]['add']!='Denied':
                newConfirmed[req]=add.value[req]['data']
            elif add.value[req]['add']=='Denied':
                newDenied[req]=add.value[req]['data']
                
        for req in edit.value:
            if edit.value[req]['add']!='Denied':
                newConfirmed[req]=edit.value[req]['data']
                if req in newDenied:
                    del(newDenied[req])
            elif edit.value[req]['add']=='Denied':
                newDenied[req]=edit.value[req]['data']
                if req in newConfirmed:
                    del(newConfirmed[req])
        newconfirmedFile=open(pathway+datasetName+'Confirmed','w+b')
        newdeniedFile=open(pathway+datasetName+'Denied','w+b')
        pkl.dump(newConfirmed,newconfirmedFile)
        pkl.dump(newDenied,newdeniedFile)
        newconfirmedFile.close()
        newdeniedFile.close()
        confirmed.value=newConfirmed
        denied.value=newDenied
        add.value={}
        edit.value={}
        
        
    def submitNameFunc(b,userNameValue=userNameValue,groupcounter=groupcounter,batchcounter=batchcounter):
        bad='1234567890-=[]\\;\',./!@#$%^&*()_+{}|:\"<>? '
        if userName.value and not any(x in userName.value for x in bad):
            print('Setting User to',userName.value)
            userNameValue.changeName(userName.value)
            curLabel.value="User: "+userNameValue.value+", Working on: Group: "+str(groupcounter.value)+", Batch: "+str(batchcounter.value)
            b.style.button_color='green'
        else:
            print('Empty input or not alphabetic found in input')
            b.style.button_color='red'
    
    def loadNextGroup(b,groupcounter=groupcounter,batchcounter=batchcounter):
        if (groupcounter.value < len(groups.keys())):
            #Reset map
            m = folium.Map()
            
            #Check if Batch # is larger than it should be and move onto the next group
            if (batchcounter.value >= math.ceil(len(groups[str(groupcounter.value)])/IMGSHOWN)):
                groupcounter.increment()
                batchcounter.reset()
            
            curLabel.value="User: "+userNameValue.value+", Working on: Group: "+str(groupcounter.value)+", Batch: "+str(batchcounter.value)
            curBatch.clear_output()
            with curBatch:
                display(curLabel)
                
            #Print current grouping information for debugging/early termination purposes
            print('group #')
            print(groupcounter.value)
            print('batch #')
            print(batchcounter.value)
            print('gjs group batch #')
            print(len(groupGJS[groupcounter.value]))
            print('group size')
            print(len(groupImg[groupcounter.value]))
            
            #Add the relevant group-batch gjs to the map
            for gjs in groupGJS[groupcounter.value][batchcounter.value]:
                m.add_layer(gjs)
            
            #Initialize the group-batch images from the list and display it using a clearable output to effectively 'refresh'
            horizontalBox=VBox([groupImg[groupcounter.value][batchcounter.value]])
            
            HiImg.clear_output()
            with HiImg:
                display(horizontalBox)
            
            #Increase step counter!
            batchcounter.increment()
            
    #Tie next group button to it's on-click event
    saveData.on_click(functools.partial(saveDataFunc,confirmedClass,deniedClass,add,edit))
    nextGroup.on_click(loadNextGroup)
    submitName.on_click(submitNameFunc)

    #create custom box layout for the future image section, map and buttons
    HiImg=Output(layout={'border': '1px solid black'})
    horizontalBox=Box([HiImg],layout=Layout(
        display='flex',
        flex_flow='row',
        width='100%'
    ))
    curBatch=Output(layout={'border': '1px solid black'})
    with curBatch:
        display(curLabel)
    mapOutput=Output(layout={'border':'1px solid black'})
    with mapOutput:
        display(m)
    horizontalBox1=Box([mapOutput,curBatch],layout=Layout(
        display='flex',
        flex_flow='column',
        width='100%'
    ))
    
    #create actual layout for the UI
    maps=VBox([horizontalBox1,horizontalBox])
    button=HBox([nextGroup,saveData,userName,submitName])
    total=VBox([maps,button])
    
    #Return the callable UI!
    return total


## Look over these next cells and edit/change or initialize any necessary values

In [5]:
#Initialize the basic variables
if not os.path.exists(pathway):
    os.mkdir(pathway)
if not os.path.exists(pathway+datasetName+'Confirmed'):
    confirmed={}
    confirmedFile=open(pathway+datasetName+'Confirmed','w+b')
    pkl.dump(confirmed,confirmedFile)
    confirmedFile.close()
else:
    confirmedFile=open(pathway+datasetName+'Confirmed','r+b')
    confirmed=pkl.load(confirmedFile)
    confirmedFile.close()
if not os.path.exists(pathway+datasetName+'Denied'):
    denied={}
    deniedFile=open(pathway+datasetName+'Denied','w+b')    
    pkl.dump(denied,deniedFile)
    deniedFile.close()
else:
    deniedFile=open(pathway+datasetName+'Denied','r+b')    
    denied=pkl.load(deniedFile)
    deniedFile.close()
batch=0

In [6]:
print(confirmed)

{}


## Run the next two cells as many times as necessary to iterate through all of the batches

In [7]:
print("Batch #",batch)
groupfile=open(pathway+datasetName+'GroupBatch'+str(batch),'r+b')
group=pkl.load(groupfile)
groupfile.close()
UI=InitiateUI(confirmed,denied,group,batch)
batch+=1

Batch # 0
Loading Data
Initializing UI!
Adding group # 0  to the web app
Adding group # 1  to the web app
Adding group # 2  to the web app
Adding group # 3  to the web app
Done!


In [8]:
#Call the result to display in the below cell, continue until images no longer load.
UI

In [9]:
#Print current confirmed,denied list to ensure update
finalconfirmedfile=open(pathway+datasetName+'Confirmed','rb')
finaldeniedfile=open(pathway+datasetName+'Denied','rb')
finalgroups=open(pathway+datasetName+'GroupBatch'+str(0),'rb')
confirmed=pkl.load(finalconfirmedfile)
denied=pkl.load(finaldeniedfile)
groups=pkl.load(finalgroups)
finalconfirmedfile.close()
finaldeniedfile.close()
finalgroups.close()
tally=0
for x in groups:
    tally+=len(groups[x])
print(len(confirmed))
print(len(denied))
print(tally)

474
744
1221


In [10]:
print('hi')

hi
